# Sprint 深層学習スクラッチ 畳み込みニューラルネットワーク２

<h1>目次<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#このSprintについて" data-toc-modified-id="このSprintについて-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>このSprintについて</a></span><ul class="toc-item"><li><span><a href="#Sprintの目的" data-toc-modified-id="Sprintの目的-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Sprintの目的</a></span></li><li><span><a href="#どのように学ぶか" data-toc-modified-id="どのように学ぶか-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>どのように学ぶか</a></span></li></ul></li><li><span><a href="#2次元の畳み込みニューラルネットワークスクラッチ" data-toc-modified-id="2次元の畳み込みニューラルネットワークスクラッチ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2次元の畳み込みニューラルネットワークスクラッチ</a></span><ul class="toc-item"><li><span><a href="#データセットの用意" data-toc-modified-id="データセットの用意-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>データセットの用意</a></span></li><li><span><a href="#【問題1】2次元畳み込み層の作成" data-toc-modified-id="【問題1】2次元畳み込み層の作成-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>【問題1】2次元畳み込み層の作成</a></span></li><li><span><a href="#【問題2】2次元畳み込み後の出力サイズ" data-toc-modified-id="【問題2】2次元畳み込み後の出力サイズ-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>【問題2】2次元畳み込み後の出力サイズ</a></span></li><li><span><a href="#【問題3】最大プーリング層の作成" data-toc-modified-id="【問題3】最大プーリング層の作成-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>【問題3】最大プーリング層の作成</a></span></li><li><span><a href="#【問題4】（アドバンス課題）平均プーリングの作成" data-toc-modified-id="【問題4】（アドバンス課題）平均プーリングの作成-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>【問題4】（アドバンス課題）平均プーリングの作成</a></span></li><li><span><a href="#【問題5】平滑化" data-toc-modified-id="【問題5】平滑化-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>【問題5】平滑化</a></span></li><li><span><a href="#【問題6】学習と推定" data-toc-modified-id="【問題6】学習と推定-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>【問題6】学習と推定</a></span></li><li><span><a href="#【問題7】（アドバンス課題）LeNet" data-toc-modified-id="【問題7】（アドバンス課題）LeNet-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>【問題7】（アドバンス課題）LeNet</a></span></li></ul></li></ul></div>

## このSprintについて
### Sprintの目的
* スクラッチを通してCNNの基礎を理解する

### どのように学ぶか
スクラッチで2次元用畳み込みニューラルネットワークを実装した後、学習と検証を行なっていきます。

## 2次元の畳み込みニューラルネットワークスクラッチ
2次元に対応した畳み込みニューラルネットワーク（CNN）のクラスをスクラッチで作成していきます。  
NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。  
  
プーリング層なども作成することで、CNNの基本形を完成させます。  
クラスの名前はScratch2dCNNClassifierとしてください。

### データセットの用意
引き続きMNISTデータセットを使用します。2次元畳み込み層へは、28×28の状態で入力します。

今回は白黒画像ですからチャンネルは1つしかありませんが、チャンネル方向の軸は用意しておく必要があります。

(n_samples, n_channels, height, width)のNCHWまたは(n_samples, height, width, n_channels)のNHWCどちらかの形にしてください。

In [24]:
# インポート
import numpy as np
import matplotlib.pyplot as plt

In [31]:
# MNISTデータセットのダウンロード
from keras.datasets import mnist
(X, y), (X_test, y_test) = mnist.load_data()

In [32]:
# データの確認
print(X.shape) # (60000, 28, 28)
print(X.shape) # (10000, 28, 28)
print(X[0].dtype) # uint8

(60000, 28, 28)
(60000, 28, 28)
uint8


### 【問題1】2次元畳み込み層の作成
1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。

フォワードプロパゲーションの数式は以下のようになります。

$$a_{i,j,m} = \sum_{k=0}^{K-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1}x_{(i+s),(j+t),k}w_{s,t,k,m}+b_{m}$$

In [224]:
# 1次元畳み込み層クラス
class SimpleConv2d():
    """
    2次元畳み込み層
    Parameters
    ----------
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, N, C, H, W, F, FH, FW, P, S,
                 initializer=None,optimizer=None,activation=None):
        self.P = P
        self.S = S
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = self.initializer.W(F,C,FH,FW)
        self.B = self.initializer.B(F)
        
    def output_shape2d(IH=5,IW=5,PH=0,PW=0,FH=3,FW=3,SH=1,SW=1):
        OH = (IH +2*PH -FH)/SH +1
        OW = (IW +2*PW -FW)/SW +1
        return int(OH),int(OW)
    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.X = X
        N,C,H,W = self.X.shape
        F,C,FH,FW = self.W.shape
        
        OH,OW = output_shape2d(H,W,self.P,self.P,FH,FW,self.S,self.S)

        A = np.zeros([N,F,OH,OW])

        self.X_pad = np.pad(X,((0,0),(0,0),(P,P),(P,P)))

        # バッチ
        for n in range(N):
            # 出力チャンネル
            for ch in range(F):
                # 縦方向のスライド
                for row in range(0,H,S):
                    # 横方向のスライド
                    for col in range(0,W,S):
                        A[n,ch,row,col] = \
                        np.sum(self.X_pad[n,:,row:row+FH,col:col+FW]*self.W[ch,:,:,:]) +self.B[ch]
        
        return  self.activation.forward(A)
    
    def backward(self, dZ):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ = np.zeros(self.X_pad.shape)
        self.dW = np.zeros(self.W.shape)
        self.dB = np.zeros(self.B.shape)
        
        N,C,H,W = self.X.shape
        F,C,FH,FW = self.W.shape
        OH,OW = output_shape2d(H,W,self.P,self.P,FH,FW,self.S,self.S)
        
        
        # dZ
        # バッチ
        for n in range(N):
            # 出力チャンネル
            for ch in range(F):
                # 縦方向のスライド
                for row in range(0,H,S):
                    # 横方向のスライド
                    for col in range(0,W,S):
                        dZ[n,:,row:row+FH,col:col+FW] += dA[n,ch,row,col]*w[ch,:,:,:]
                
        dl_rows = range(P),range(H+P,H+2*P,1)
        dl_cols = range(P),range(W+P,W+2*P,1)

        dZ = np.delete(dZ,dl_rows,axis=2)
        dZ = np.delete(dZ,dl_cols,axis=3)
                
        # dW
        # バッチ
        for n in range(N):
            # 出力チャンネル
            for ch in range(F):
                # 縦方向のスライド
                for row in range(OH):
                    # 横方向のスライド
                    for col in range(OW):
                        self.dW[ch,:,:,:] += dA[n,ch,row,col]*X_pad[n,:,row:row+FH,col:col+FW]
        
        # dB
        # 出力チャンネル
        for ch in range(F):
            self.dB[ch] = np.sum(dA[:,ch,:,:])
        
        # 更新
        self = self.optimizer.update(self)
        
        return dZ

In [223]:
class SimpleInitializerConv2d:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma=0.01):
        self.sigma = sigma
        
    def W(self, F, C, FH, FW):
        """
        重みの初期化
        Parameters
        ----------
        
        Returns
        ----------
        W : 重み
        """
        return self.sigma * np.random.randn(F,C,FH,FW)
    
    def B(self, F):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B : バイアス
        """
        return np.zeros(F)

In [226]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.W -= self.lr*layer.dW
        layer.B -= self.lr*layer.dB
        
        return layer

In [227]:
class ReLU():
    """
    活性化関数 : ReLU関数
    """
    def __init__(self):
        pass
        
    def forward(self,A):
        self.A = A
        return np.maximum(self.A,0)
    
    def backward(self,dZ):
        
        return np.where(self.A>0,dZ,0)

### 【問題2】2次元畳み込み後の出力サイズ
畳み込みを行うと特徴マップのサイズが変化します。  
どのように変化するかは以下の数式から求められます。  
この計算を行う関数を作成してください。

In [26]:
def output_shape2d(IH=5,IW=5,PH=0,PW=0,FH=3,FW=3,SH=1,SW=1):
    OH = (IH +2*PH -FH)/SH +1
    OW = (IW +2*PW -FW)/SW +1
    return int(OH),int(OW)

In [88]:
print(output_shape2d(IH=6,IW=6,PH=0,PW=0,FH=3,FW=3,SH=1,SW=1))

(4, 4)


In [85]:
N,C,H,W = (5,1,28,28)
F,C,FH,FW = (4,1,3,3)

S = 1 #とりあえず固定
P = 1

OH,OW = output_shape2d(H,W,P,P,FH,FW,S,S)

A = np.zeros([N,F,OH,OW])

X_sample = X[0:N].reshape(N,C,H,W)
X_pad = np.pad(X_sample,((0,0),(0,0),(P,P),(P,P)))
w = np.ones([F,C,FH,FW])
B = np.ones(F)

# フォワード

# バッチ
for n in range(N):
    # 出力チャンネル
    for ch in range(F):
        # 縦方向のスライド
        for row in range(0,H,S):
            # 横方向のスライド
            for col in range(0,W,S):
                A[n,ch,row,col] = \
                np.sum(X_pad[n,:,row:row+FH,col:col+FW]*w[ch,:,:,:]) +B[ch]
                
print('A.shape:',A.shape)

A.shape: (5, 4, 28, 28)


In [221]:
# バックワード
dA = np.ones(A.shape)

dZ = np.zeros(X_pad.shape)
dW = np.zeros(w.shape)
dB = np.zeros(B.shape)

# dZ
# バッチ
for n in range(N):
    # 出力チャンネル
    for ch in range(F):
        # 縦方向のスライド
        for row in range(0,H,S):
            # 横方向のスライド
            for col in range(0,W,S):
                dZ[n,:,row:row+FH,col:col+FW] += dA[n,ch,row,col]*w[ch,:,:,:]
                
dl_rows = range(P),range(H+P,H+2*P,1)
dl_cols = range(P),range(W+P,W+2*P,1)

dZ = np.delete(dZ,dl_rows,axis=2)
dZ = np.delete(dZ,dl_cols,axis=3)
                
# dW
# バッチ
for n in range(N):
    # 出力チャンネル
    for ch in range(F):
        # 縦方向のスライド
        for row in range(OH):
            # 横方向のスライド
            for col in range(OW):
                dW[ch,:,:,:] += dA[n,ch,row,col]*X_pad[n,:,row:row+FH,col:col+FW]
                
# dB
# 出力チャンネル
for ch in range(F):
    dB[ch] = np.sum(dA[:,ch,:,:])
                
print('dZ.shape:',dZ.shape)
print('dW.shape:',dW.shape)
print('dB.shape:',dB.shape)

dZ.shape: (5, 1, 28, 28)
dW.shape: (4, 1, 3, 3)
dB.shape: (4,)


### 【問題3】最大プーリング層の作成
最大プーリング層のクラスMaxPool2Dを作成してください。  
プーリング層は数式で表さない方が分かりやすい部分もありますが、数式で表すとフォワードプロパゲーションは以下のようになります。

In [230]:
class MaxPool2D:
    def __ini__(self,PS=2):
        self.PS = PS
        self.PA = None
        self.Pindex = None
        
    def forward(self,N,F,OH,OW):
        # プーリング
        self.PA = np.zeros([N,F,int(OH/PS),int(OW/PS)])
        self.Pindex = np.zeros([N,F,int(OH/PS),int(OW/PS)])
        
        for n in range(N):
            # 出力チャンネル
            for ch in range(F):
                # 縦方向のスライド
                for row in range(0,OH,PS):
                    # 横方向のスライド
                    for col in range(0,OW,PS):
                        self.PA[n,ch,int(row/PS),int(col/PS)] = \
                        np.max(A[n,ch,row:row+PS,col:col+PS])
                        
                        self.Pindex[n,ch,int(row/PS),int(col/PS)] = \
                        np.argmax(A[n,ch,row:row+PS,col:col+PS])
                        
        return self.PA
    
    def backward(self,dA):
        dP = np.zeros(A.shape)
        
        for n in range(N):
            # 出力チャンネル
            for ch in range(F):
                # 縦方向のスライド
                for row in range(0,OH,PS):
                    # 横方向のスライド
                    for col in range(0,OW,PS):
                        idx = int(Pindex[n,ch,int(row/PS),int(col/PS)])
                        tmp = np.zeros((PS*PS))
                        for i,j in enumerate(dA[n,ch,row:row+PS,col:col+PS].reshape(-1,)):
                            if i == idx:
                                tmp[i] = j
                            else:
                                tmp[i] = 0
                        tmp = tmp.reshape(PS,PS)
                        dP[n,ch,row:row+PS,col:col+PS] = tmp
        
        return dP
        

In [231]:
# プーリングのサイズ、ストライド
PS = 4

# プーリング
PA = np.zeros([N,F,int(OH/PS),int(OW/PS)])
Pindex = np.zeros([N,F,int(OH/PS),int(OW/PS)])

# フォワード
# バッチ
for n in range(N):
    # 出力チャンネル
    for ch in range(F):
        # 縦方向のスライド
        for row in range(0,OH,PS):
            # 横方向のスライド
            for col in range(0,OW,PS):
                PA[n,ch,int(row/PS),int(col/PS)] = np.max(A[n,ch,row:row+PS,col:col+PS])
                Pindex[n,ch,int(row/PS),int(col/PS)] = np.argmax(A[n,ch,row:row+PS,col:col+PS])
                
                #print(A[n,ch,row:row+PS,col:col+PS])
                #print(np.argmax(A[n,ch,row:row+PS,col:col+PS]))
                      
print('PA.shape:\n',PA.shape)
print('Pindex.shape:\n',Pindex[0,0,0,0])

PA.shape:
 (5, 4, 7, 7)
Pindex.shape:
 0.0


In [234]:
dP = np.zeros(A.shape)
dA = np.ones(A.shape)

# バックワード
# バッチ
for n in range(N):
    # 出力チャンネル
    for ch in range(F):
        # 縦方向のスライド
        for row in range(0,OH,PS):
            # 横方向のスライド
            for col in range(0,OW,PS):
                idx = int(Pindex[n,ch,int(row/PS),int(col/PS)])
                tmp = np.zeros((PS*PS))
                for i,j in enumerate(dA[n,ch,row:row+PS,col:col+PS].reshape(-1,)):
                    if i == idx:
                        tmp[i] = j
                    else:
                        tmp[i] = 0
                tmp = tmp.reshape(PS,PS)
                dP[n,ch,row:row+PS,col:col+PS] = tmp
                
print('dP:\n',dP[0,0,0,:])

dP:
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0.]


### 【問題4】（アドバンス課題）平均プーリングの作成
平均プーリング層のクラスAveragePool2Dを作成してください。

範囲内の最大値ではなく、平均値を出力とするプーリング層です。

画像認識関係では最大プーリング層が一般的で、平均プーリングはあまり使われません。

### 【問題5】平滑化
平滑化するためのFlattenクラスを作成してください。

フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。

この平滑化のクラスを挟むことで出力前の全結合層に適した配列を作ることができます。

In [211]:
class Flatten:
    def __ini__(self,):
        pass
    def forward(self,X):
        self.shape = X.shape
        return X.reshape(len(X),-1)

    def backward(self,X):
        return X.reshape(self.shape)        

In [218]:
TEST = np.zeros([20,2,5,5])
flt = Flatten()
flat_forward = flt.forward(TEST)
print('Forward_shape:',flat_forward.shape)
print('Backward_shape:',flt.backward(flat_forward).shape)

Forward_shape: (20, 50)
Backward_shape: (20, 2, 5, 5)


### 【問題6】学習と推定
作成したConv2dを使用してMNISTを学習・推定し、Accuracyを計算してください。

精度は低くともまずは動くことを目指してください。

In [228]:
conv2d = SimpleConv2d(N=5, C=1, H=28, W=28, F=4, FH=3, FW=3, P=1, S=1,
                      initializer=SimpleInitializerConv2d(),
                      optimizer=SGD(),
                      activation=ReLU())

In [229]:
conv2d.forward(X_sample)

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 

In [ ]:
# スクラッチ　CNN
class Scratch2dCNNClassifier():
    """
    N層の畳み込みニューラルネットワーク分類器
    
    Parameters
    ----------
    self.n_epoch : エポック数
    self.n_batch : バッチ数
    self.verbose : 学習過程を可視化
    Attributes
    ----------
    """
    def __init__(self, NN, n_epoch=5, n_batch=1, verbose = False):
        #　パラメータ
        self.n_epoch = n_epoch
        self.n_batch = n_batch
        self.verbose = verbose
        self.log_loss = np.zeros(self.n_epoch)
        self.log_acc = np.zeros(self.n_epoch)
        self.NN = NN
        
    def loss_function(self,y,yt):
        delta = 1e-7
        return -np.mean(yt*np.log(y+delta))
    
    def accuracy(self,Z,Y):
        return accuracy_score(Y,Z)
                
    def fit(self, X, y, X_val=False, y_val=False):
        """
        ニューラルネットワーク分類器を学習する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        for epoch in range(self.n_epoch):
            # ミニバッチ処理
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.n_batch)
            
            self.loss = 0
            for mini_X_train, mini_y_train in get_mini_batch:
                
                # 順伝播
                forward_data = mini_X_train
                forward_data = conv.forward(forward_data)
                forward_data = forward_data.reshape(self.n_batch,-1)
                
                for layer in range(len(self.NN)):
                    forward_data = self.NN[layer].forward(forward_data)
                    
                # 予測値
                Z = forward_data
                
                # 逆伝播
                backward_data = (Z - mini_y_train)/self.n_batch
                for layer in range(len(self.NN)-1,-1,-1):
                    backward_data = self.NN[layer].backward(backward_data)
                
                conv.backward(backward_data.reshape(self.n_batch,100,782))
                
                # 損失関数
                self.loss += self.loss_function(Z,mini_y_train)
                
            self.log_loss[epoch] = self.loss/len(get_mini_batch)
            self.log_acc[epoch] = self.accuracy(self.predict(X),np.argmax(y,axis=1))
            
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        y_pred = np.zeros(len(X))
        for i,pred in enumerate(X):
            pred_data = pred.reshape(1,-1)
            pred_data = conv.forward(pred_data)
            pred_data = pred_data.reshape(1,-1)
        
            for layer in range(len(self.NN)):
                pred_data = self.NN[layer].forward(pred_data)
                y_pred[i] = np.argmax(pred_data,axis=1)
        return y_pred

### 【問題7】（アドバンス課題）LeNet
CNNで画像認識を行う際は、フィルタサイズや層の数などを１から考えるのではなく、有名な構造を利用することが一般的です。現在では実用的に使われることはありませんが、歴史的に重要なのは1998年の LeNet です。この構造を再現してMNISTに対して動かし、Accuracyを計算してください。